In [4]:
from common import *

cursor = connect()

# Materialized View

物化视图以物理方式存储查询结果，并定期从基础表中刷新数据。物化视图可以提高查询性能，但是会增加数据更新的复杂性。

```
CREATE MATERIALIZED VIEW [IF NOT EXISTS] view_name
AS
query
WITH [NO] DATA;
```

## 刷新物化视图的数据
```
REFRESH MATERIALIZED VIEW view_name;
```

为物化视图刷新数据时，PostgreSQL 会锁定底层表。因此，当数据载入视图时，将无法从底层表中检索数据。为了避免这种情况，可以使用 `CONCURRENTLY` 选项。
```
REFRESH MATERIALIZED VIEW CONCURRENTLY view_name;
```
使用 CONCURRENTLY 选项，PostgreSQL 会创建一个物化视图的临时更新版本，比较两个版本，并只对差异执行 INSERT 和 UPDATE。PostgreSQL 允许在更新物化视图时从该视图中检索数据。使用 CONCURRENTLY 选项的一个要求是，物化视图必须有一个 UNIQUE 索引。

## 删除物化视图
```
DROP MATERIALIZED VIEW view_name;
```

In [5]:
sql = """
CREATE MATERIALIZED VIEW rental_by_category
AS
 SELECT c.name AS category,
    sum(p.amount) AS total_sales
   FROM (((((payment p
     JOIN rental r ON ((p.rental_id = r.rental_id)))
     JOIN inventory i ON ((r.inventory_id = i.inventory_id)))
     JOIN film f ON ((i.film_id = f.film_id)))
     JOIN film_category fc ON ((f.film_id = fc.film_id)))
     JOIN category c ON ((fc.category_id = c.category_id)))
  GROUP BY c.name
  ORDER BY sum(p.amount) DESC
WITH NO DATA;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost port=15432 user=postgres database=dvdrental) at 0x1b4d9728880>

In [3]:
# 因为使用了 `WITH NO DATA`，所以无法从视图中查询数据。
sql = """
SELECT * FROM rental_by_category;
"""
run_sql(cursor, sql)

ObjectNotInPrerequisiteState: 物化视图 "rental_by_category"未被初始化
HINT:  使用命令 REFRESH MATERIALIZED VIEW.

In [6]:
# 刷新物化视图的数据
sql = """
REFRESH MATERIALIZED VIEW rental_by_category;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost port=15432 user=postgres database=dvdrental) at 0x1b4d9728880>

In [7]:
# 因为使用了 `WITH NO DATA`，所以无法从视图中查询数据。
sql = """
SELECT * FROM rental_by_category;
"""
run_sql(cursor, sql)

       category total_sales
0        Sports     4892.19
1        Sci-Fi     4336.01
2     Animation     4245.31
3         Drama     4118.46
4        Comedy     4002.48
5           New     3966.38
6        Action     3951.84
7       Foreign     3934.47
8         Games     3922.18
9        Family     3830.15
10  Documentary     3749.65
11       Horror     3401.27
12     Classics     3353.38
13     Children     3309.39
14       Travel     3227.36
15        Music     3071.52


In [8]:
# 为了使用 ``CONCURRENTLY`` 选项，物化视图必须有一个 UNIQUE 索引。
sql = """
CREATE UNIQUE INDEX rental_category 
ON rental_by_category (category);
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost port=15432 user=postgres database=dvdrental) at 0x1b4d9728880>

In [9]:
sql = """
REFRESH MATERIALIZED VIEW CONCURRENTLY rental_by_category;
"""
cursor.execute(sql)

<psycopg.Cursor [COMMAND_OK] [INTRANS] (host=localhost port=15432 user=postgres database=dvdrental) at 0x1b4d9728880>